# Iterator

一つ一つ数え上げる

## Iteratorパターン

複数の要素から成る配列を先頭から終端まで走査する場合、普通に実装すると以下のような形になる

In [1]:
const arr: string[] = ['A', 'B', 'C'];
for (let i: number = 0; i < arr.length; ++i) {
    console.log(arr[i]);
}

Configuration is not found! Default configuration will be used: {"module":1,"target":1,"moduleResolution":2,"esModuleInterop":true}

A
B
C


すなわち、添字を 0〜(配列の長さ-1) までインクリメントしていくことで、配列全体を走査することになる

このような走査パターンをIteratorパターンに抽象化することで、配列以外のあらゆる集合体に対して適用できるようになる

***

## サンプルプログラム

本棚（BookShelf）の中に本（Book）を入れ、その本の名前を順番に表示する

![iterator](./img/iterator.svg)

### Aggregateインタフェース: 集合体
このインタフェースは、数え上げを行うものの「集合体」を表す

Aggregateは、iteratorメソッドのみを持つ

iteratorメソッドは、集合体に対応するIteratorを1つ作成する

集合体を1つずつスキャンしたい場合は、このiterateメソッドを使ってIteratorインタフェースを実装したクラスのインスタンスを作る

### Iteratorインタフェース: 反復子
このインタフェースは、要素の数え上げを行うループ変数のような役割を果たす

ここでは、次の要素が存在するかどうかを判定するhasNextメソッドと、次の要素を得るためのnextメソッドを定義しておく

- hasNext:
    - 次の要素が存在する場合はtrueを返す
- next:
    - 現在のポインタが指し示す集合体の要素を1つ返し、次の要素にポインタを一つ進める

### Bookクラス
このクラスは本を表すクラスであり、本の名前をgetNameメソッドで取得することができる

### BookShelfクラス: ConcreteAggregate（集合体のインスタンス）
このクラスは本棚を表すクラスであり、本の集合体として扱うためにAggregateインタフェースを実装する

本棚は、Bookオブジェクトの配列をフィールドとして所有し、この配列をスキャンする

### BookShelfIterator: ConcreteIterator（反復子のインスタンス）
このクラスはBookShelfクラスのスキャンを行うクラスであり、Iteratorインタフェースを実装する

In [2]:
/**
 * @interface Iterator
 * 要素の数え上げを行う反復子
 */
interface Iterator {
    // 次の要素が存在するか判定
    hasNext(): boolean;
    
    // 現在のポインタが指し示す集合体の要素を1つ返し、次の要素にポインタを一つ進める
    next(): object;
}

/**
 * @interface Aggregate
 * 数え上げ対象の集合体
 */
interface Aggregate {
    // Iteratorのインスタンスを生成
    iterator(): Iterator;
}

/**
 * @class Book
 * 集合体の要素（本）
 */
class Book {
    private name: string;
    
    public constructor(name: string) {
        this.name = name;
    }
    
    public getName(): string {
        return this.name;
    }
}

/**
 * @class BookShelf <: Aggregate
 * 数え上げ対象（本）の集合体（本棚）
 */
class BookShelf implements Aggregate {
    private books: Book[];
    private last: number;
    
    public constructor(maxLength: number) {
        this.books = new Array(maxLength);
        this.last = 0;
    }
    
    public getBookAt(index: number): Book {
        return this.books[index];
    }
    
    public appendBook(book: Book): void {
        this.books[this.last++] = book;
    }
    
    public getLength(): number {
        return this.last;
    }
    
    // Iteratorのインスタンスを生成
    public iterator(): BookShelfIterator {
        return new BookShelfIterator(this);
    }
}

/**
 * @class BookShelfIterator <: Iterator
 * 要素（本）の数え上げを行う反復子
 */
class BookShelfIterator implements Iterator {
    private bookShelf: BookShelf;
    private index: number;
    
    public constructor(bookShelf: BookShelf) {
        this.bookShelf = bookShelf;
        this.index = 0;
    }
    
    // 次の要素が存在するか判定
    public hasNext(): boolean {
        return this.index < this.bookShelf.getLength();
    }
    
    // 現在のポインタが指し示す集合体の要素を1つ返し、次の要素にポインタを一つ進める
    public next(): Book {
        return this.bookShelf.getBookAt(this.index++);
    }
}

undefined

In [4]:
/**
 * Main: 4冊の本を本棚に収納し、本棚の反復子を用いて全体をスキャンする
 */
const bookShelf: BookShelf = new BookShelf(4); // 本棚(4冊分)

bookShelf.appendBook(new Book('ゼロから作るDeepLearning'));
bookShelf.appendBook(new Book('デザインパターン入門'));
bookShelf.appendBook(new Book('44の例題で学ぶ計量経済学'));
bookShelf.appendBook(new Book('SOFT SKILLS'));

const it: BookShelfIterator = bookShelf.iterator(); // 本棚の反復子

// 次の要素がある限りループ
while(it.hasNext()) {
    console.log(it.next().getName());
}

ゼロから作るDeepLearning
デザインパターン入門
44の例題で学ぶ計量経済学
SOFT SKILLS


undefined